# SSE Report
***Note*** This script is for SSE's bug report. Input the `.db` file of SSE and then digest the `.db` file by some filter options into a more readable text form.

In [1]:
import sqlite3
import re as re

In [2]:
# open the db file
def open_db_file(db_filepath):
    db = sqlite3.connect(db_filepath)
    return db

In [3]:
# fetch all the cl and ln
def get_location(db):
    cur = db.cursor()
    res = cur.execute("SELECT location, status FROM checks")
    res_all = res.fetchall()
    return res_all

In [4]:
def get_ln_cl_list_sse(loc):
    loc_list = []
    for check_loc in loc:
        loc_str = check_loc[0]
        search_res = re.search("ln: (\d*)[ ]*cl: (\d*)", loc_str)
        if search_res != None:
            loc_list.append((int(search_res[1]), int(search_res[2]), check_loc[1]))
        else:
            loc_list.append((-1, -1, 0))
    return loc_list

In [11]:
def get_ln_cl_list_ikos(db):
    cur = db.cursor()
    res = cur.execute("SELECT s.line, s.column, c.status FROM checks as c INNER JOIN statements as s WHERE s.id = c.statement_id AND c.status IN(0, 1, 2)")
    res_all = res.fetchall()
    return res_all

In [6]:
db_sse = open_db_file("output.db")
db_ikos = open_db_file("output_ikos.db")

In [7]:
loc = get_location(db_sse)

In [8]:
get_ln_cl_list_sse(loc)

[(-1, -1, 0), (3, 5, -2)]

In [12]:
ikos_len = get_ln_cl_list_ikos(db_ikos)

In [13]:
ikos_len

[(12, 19, 0),
 (12, 14, 0),
 (12, 14, 0),
 (12, 14, 0),
 (12, 14, 0),
 (2, 5, 0),
 (2, 12, 0),
 (2, 12, 0),
 (2, 12, 0),
 (3, 12, 0),
 (3, 12, 0),
 (3, 12, 0),
 (3, 12, 0),
 (3, 5, 0),
 (13, 5, 0)]

In [34]:
def merge_as_highest_level(res_list):
    loc_status_map = {}
    for res in res_list:
        loc = (res[0], res[1])
        if loc in loc_status_map:
            loc_status_map[loc] = max(res[2], loc_status_map[loc])
            print("reset")
        else:
            loc_status_map[loc] = res[2]
            print("init")
    return loc_status_map

In [35]:
merged = merge_level(ikos_len)

init
init
reset
reset
reset
init
init
reset
reset
init
reset
reset
reset
init
init


In [36]:
merged

{(12, 19): 0,
 (12, 14): 0,
 (2, 5): 0,
 (2, 12): 0,
 (3, 12): 0,
 (3, 5): 0,
 (13, 5): 0}

In [1]:
import json
with open("juliet_testcases/metadata/CWE121_Stack_Based_Buffer_Overflow__CWE135_0.sarif", "r") as f:
    metadata = json.load(f)

In [31]:
metadata['runs'][0]['results'][1]

{'ruleId': 'CWE-121',
 'message': {'text': 'Stack-based Buffer Overflow.'},
 'locations': [{'physicalLocation': {'artifactLocation': {'uri': 'src/testcases/CWE121_Stack_Based_Buffer_Overflow/s01/CWE121_Stack_Based_Buffer_Overflow__CWE135_09.c',
     'index': 0},
    'region': {'startLine': 42}}}],
 'taxa': [{'toolComponent': {'name': 'CWE', 'index': 0},
   'id': '121',
   'index': 0}]}